In [1]:
import os
import argparse
import random
import subprocess
import numpy as np
import yaml
import torch.multiprocessing as mp
import torch.multiprocessing

torch.multiprocessing.set_sharing_strategy('file_system')

def set_cuda_visible_devices():

    if "CUDA_VISIBLE_DEVICES" not in os.environ:
        gpu_ids = input('Please select the GPU IDs (e.g., 0,1,2 for 3 GPUs): ')
        
        clean_ids = [g.strip() for g in gpu_ids.split(',') if g.strip()]
        if not clean_ids:
            print("No valid GPU ID provided. Exiting.")
            exit(1)
            
        os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(clean_ids)
        num_gpus_to_use = len(clean_ids)
        print(f"Set CUDA_VISIBLE_DEVICES={os.environ['CUDA_VISIBLE_DEVICES']}. Will use {num_gpus_to_use} GPU(s).")
        return num_gpus_to_use
    else:
        gpu_ids = os.environ["CUDA_VISIBLE_DEVICES"]
        clean_ids = [g.strip() for g in gpu_ids.split(',') if g.strip()]
        return len(clean_ids)

In [2]:
try:
    WORLD_SIZE_FROM_USER = set_cuda_visible_devices()
except Exception as e:
    print(f"Error during GPU selection: {e}")
    exit(1)

Please select the GPU IDs (e.g., 0,1,2 for 3 GPUs): 0
Set CUDA_VISIBLE_DEVICES=0. Will use 1 GPU(s).


In [3]:
from train import SimCLR
from dataloader.dataset_wrapper import DataSetWrapper
import torch.distributed as dist
import torch

def set_random_seed(seed, deterministic=False):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    if deterministic:
        torch.use_deterministic_algorithms(deterministic)
        os.environ['CUBLAS_WORKSPACE_CONFIG']=':4096:8'

def main(rank, world_size, rank_is_set, ds_args):
    
    if not rank_is_set:
        os.environ['MASTER_ADDR'] = 'localhost'
        os.environ['MASTER_PORT'] = '12346'
        os.environ['RANK'] = str(rank)
        os.environ["WORLD_SIZE"] = str(world_size)
        os.environ["LOCAL_RANK"] = str(rank)
        
    print(f'Process {rank}: Using local GPU index {rank} (Mapped to original ID via CUDA_VISIBLE_DEVICES)')

    torch.cuda.set_device(rank)
    torch.cuda.empty_cache()
    
    dist.init_process_group('nccl', rank=rank, world_size=world_size)
    seed_rank = torch.distributed.get_rank()
    
    device = torch.device("cuda", rank)
    seed = 2023 + seed_rank
    set_random_seed(seed, deterministic=False)
    
    config = yaml.load(open("config/config.yaml", "r"), Loader=yaml.FullLoader)
    
    dataset = DataSetWrapper(world_size, rank, config['batch_size'], **config['dataset'])

    simclr = SimCLR(dataset, config, device, world_size, rank)
    simclr.train()

/home/xieting/.conda/envs/graph-ms/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Please install apex for mixed precision training from: https://github.com/NVIDIA/apex


In [4]:
if __name__ == "__main__":
    
    os.environ["NCCL_TIMEOUT"] = "4800"
    os.environ["NCCL_BLOCKING_WAIT"] = "1"
    
    parser = argparse.ArgumentParser()
    parser.add_argument("--local_rank",
                        type=int,
                        default=-1,
                        help="local_rank for distributed training on gpus")
    parser.add_argument('--moldiff_config', type=str, default='MolDiff/configs/train/train_MolDiff.yml')
    ds_args, _ = parser.parse_known_args()

    if "WORLD_SIZE" in os.environ and "LOCAL_RANK" in os.environ:
        rank_is_set = True
    else:
        rank_is_set = False
    world_size = WORLD_SIZE_FROM_USER 
    
    if not rank_is_set:
        if world_size > 1:
            args = (world_size, rank_is_set, ds_args)
            mp.spawn(main, args=args, nprocs=world_size, join=True)
        else:
            rank = 0
            main(rank, world_size, rank_is_set, ds_args)
    else:
        rank = int(os.environ["LOCAL_RANK"])
        world_size = int(os.environ["WORLD_SIZE"])
        print('RANK', rank)
        print('WORLD_SIZE', world_size)
        main(rank, world_size, rank_is_set, ds_args)

Process 0: Using local GPU index 0 (Mapped to original ID via CUDA_VISIBLE_DEVICES)


[W1211 15:51:05.155643005 Utils.hpp:164] Warning: Environment variable NCCL_BLOCKING_WAIT is deprecated; use TORCH_NCCL_BLOCKING_WAIT instead (function operator())


calculating molecular graphs


100%|█████████████████████████████████████████████████| 8424/8424 [00:11<00:00, 709.81it/s]


Calculated 8135 molecular graph-mass spectrometry pairs
calculating molecular graphs


...

Training at Epoch 98with loss 3.069575185424711
Epoch 98


100%|████████████████████████████████████████████████████| 163/163 [00:03<00:00, 45.67it/s]


Training at Epoch 99with loss 3.0713808726679326
Epoch 99


100%|████████████████████████████████████████████████████| 163/163 [00:03<00:00, 45.15it/s]


Training at Epoch 100with loss 3.071343784683321
